In [7]:
import sys 
from scipy.interpolate import RegularGridInterpolator


sys.path.append("/users/tom/git/neon_blink_detection/")
sys.path.append("/users/tom/git/neon_blink_detection/src")

from src.features_calculator import create_grids
import numpy as np

from training.video_loader import video_loader
from src.helper import OfParams, AugParams
of_params = OfParams()
aug_params = AugParams()

clip_name = "1322-2023-02-03-14-25-41-7883a14f"

of_params = OfParams(n_layers=3, layer_interval=7, average=False, img_shape=(64, 64), grid_size=15, step_size=5, window_size=11, stop_steps=3)

rec = video_loader(of_params, aug_params)

rec._load(clip_name, bg_ratio=3, augment=False)


Number of clips: 2


In [5]:
feature_array, all_timestamps, clip_transitions = rec._load_features(
            clip_name, rec._of_params)

# Add random sampling of some indicies
n_clips = clip_transitions.shape[0] + 1
clip_feature_array = np.split(feature_array, clip_transitions + 1, axis=0)
clip_timestamps = np.split(all_timestamps, clip_transitions + 1, axis=0)

# only take y direction
feature_array = clip_feature_array[0][:, :, 1]
timestamps = clip_timestamps[0]
t = (timestamps - timestamps[0]) / 1e9

of_left = np.reshape(feature_array[:, :400], (-1,20,20))
of_right = np.reshape(feature_array[:, 400:], (-1,20,20))


Number of clips: 2


In [15]:
timestamps, eye_left_images, eye_right_images = rec._get_frames(clip_name, convert_to_gray=True)

t = (timestamps - timestamps[0]) / 1e9

# where difference in frames is larger than 100 ms
clip_transitions = np.where(np.diff(timestamps) > 1e8)[0]

n_clips = clip_transitions.shape[0] + 1
clip_left_images = np.split(eye_left_images, clip_transitions + 1)
clip_right_images = np.split(eye_right_images, clip_transitions + 1)

feature_array = []

for iclip in range(0, n_clips):

    clip_feature_array, grid = rec._compute_optical_flow(
        of_params, clip_left_images[iclip], clip_right_images[iclip]
    )

    feature_array.append(clip_feature_array)

feature_array = np.vstack(feature_array)

deprecated pixel format used, make sure you did set range correctly
 (repeated 294857 more times)
deprecated pixel format used, make sure you did set range correctly
deprecated pixel format used, make sure you did set range correctly


deprecated pixel format used, make sure you did set range correctly
 (repeated 11834 more times)
deprecated pixel format used, make sure you did set range correctly


deprecated pixel format used, make sure you did set range correctly
 (repeated 64443 more times)
deprecated pixel format used, make sure you did set range correctly


In [157]:
feature_array = feature_array[:, :, 1]

of_left = np.reshape(feature_array[:, :400], (-1,20,20))
of_right = np.reshape(feature_array[:, 400:], (-1,20,20))

In [10]:
feature_array, all_timestamps, clip_transitions = rec._load_features(
            clip_name, rec._of_params)

feature_array

Number of clips: 2


array([[[ 0.        ,  0.        ],
        [ 0.        ,  0.        ],
        [ 0.        ,  0.        ],
        ...,
        [ 0.        ,  0.        ],
        [ 0.        ,  0.        ],
        [ 0.        ,  0.        ]],

       [[ 0.        ,  0.        ],
        [ 0.        ,  0.        ],
        [ 0.        ,  0.        ],
        ...,
        [-0.16474533, -0.0471611 ],
        [-0.14723206, -0.1602974 ],
        [ 0.13716125, -0.3570366 ]],

       [[ 0.        ,  0.        ],
        [ 0.        ,  0.        ],
        [ 0.        ,  0.        ],
        ...,
        [-0.02874756,  0.08181763],
        [ 0.18031693, -0.14967346],
        [ 0.55955505, -0.6081848 ]],

       ...,

       [[ 0.04804516, -0.141963  ],
        [ 0.07432699,  0.01540279],
        [ 0.0310607 ,  0.07298088],
        ...,
        [ 0.38268661, -0.61286545],
        [ 0.5335121 , -0.5540314 ],
        [ 0.10185242, -0.34669876]],

       [[ 0.2539587 , -0.4475913 ],
        [ 0.1213541 , -0.27

In [29]:
import typing as T

def new_concatenate_features(
    feature_array: np.ndarray, of_params: OfParams, indices: np.ndarray = None
) -> np.ndarray:
    
    def get_layers(n, layer_interval):
        return np.arange(-(n // 2), (n + 1) // 2) * layer_interval
    
    interp_left, interp_right = _create_interpolater(feature_array)
    
    print(of_params)
    print(of_params.layer_interval)
    layer_interval = of_params.layer_interval / 200 # sampling rate to convert to ms

    n_frame = len(feature_array)

    if indices is None:
        indices = np.arange(n_frame) / 200

    n_grids = of_params.grid_size * of_params.grid_size * 2
    layers = get_layers(of_params.n_layers, layer_interval)

    indices_layers = np.array([[indices + i] for i in layers]).reshape(len(layers), -1)
    indices_layers = np.clip(indices_layers, 0, len(feature_array) - 1)

    features_left = np.concatenate([interpolate_spacetime(interp_left, indices, grid_size=of_params.grid_size).reshape(-1, of_params.grid_size**2) for indices in indices_layers], axis=-1)

    features_right = np.concatenate([interpolate_spacetime(interp_right, indices, grid_size=of_params.grid_size).reshape(-1, of_params.grid_size**2) for indices in indices_layers], axis=-1)

    return np.concatenate((features_left, features_right), axis=-1)

def _create_interpolater(feature_array: np.ndarray, grid_size=20):

    length = grid_size ** 2

    of_left = np.reshape(feature_array[:, :length], (-1, grid_size, grid_size))
    of_right = np.reshape(feature_array[:, length:], (-1, grid_size, grid_size))

    x = np.linspace(0, 64, grid_size, dtype=np.float32)
    y = np.linspace(0, 64, grid_size, dtype=np.float32)

    interpolator_left = RegularGridInterpolator((t, x, y), of_left, bounds_error=False, fill_value=None, method="linear")
    
    interpolator_right = RegularGridInterpolator((t, x, y), of_right, bounds_error=False, fill_value=None, method="linear")

    return interpolator_left, interpolator_right


def interpolate_spacetime(interpolator, time_points: T.List[int], grid_size: int, augmentation: T.Dict[str, int] = None):
    
    x = np.linspace(0, 64, grid_size + 2, dtype=np.float32)[1:-1]
    y = np.linspace(0, 64, grid_size + 2, dtype=np.float32)[1:-1]

    tt, xx, yy = np.meshgrid(time_points, x, y)

    txy_grid = np.concatenate((tt.reshape(-1, 1), xx.reshape(-1, 1), yy.reshape(-1, 1)), axis=1)

    if augmentation is not None:

        augm_params = get_augmentation_pars()

        grid_trans = np.zeros_like(txy_grid)

        grid_trans = txy_grid[:, 0]
        grid_trans[:, 1:] = (augm_params["linear_distort"] @ (augm_params["scale"] * (txy_grid[:, 1:] - 32.)).T).T + 32 + augm_params["translation"]

        txy_grid = grid_trans
    
    return interpolator(txy_grid).reshape(grid_size, len(time_points), grid_size).transpose(1, 0, 2)
    
def get_augmentation_pars(n_augmentations):
    
    augmentation_params = {}
    
    std_speed = 0.2
    std_translation = 3
    std_scale = 0.15
    std_linear = 0.03
    
    augmentation_params["speed"] = np.random.normal(1, std_speed, n_augmentations)
    augmentation_params["translation"] = np.random.normal(0, std_translation, (n_augmentations, 2))
    augmentation_params["scale"] = np.random.normal(1, std_scale, n_augmentations)
    augmentation_params["linear_distort"] = np.squeeze(np.dstack([np.eye(2)]*n_augmentations) + np.random.normal(0, std_linear, (2, 2, n_augmentations)))
    
    return augmentation_params


In [9]:
np.random.normal(1, 0.1)

1.0620881393362114

**Interpolate to 50x50 grid for 3 points in time**

*Plot result*

In [ ]:
# import matplotlib.pyplot as plt

# interp_left, interp_right = _create_interpolater(feature_array)

# of_interp = interpolate_spacetime(interp_left, t, grid_size=20)

# f, ax = plt.subplots(1, 2, figsize=(10, 5))

# ax[0].imshow(of_interp[5,:, :])
# ax[0].set_title("Interpolated")
                

# augm_params = get_augmentation_pars()

# p_grid = create_grids((64,64), 20 + 2, full_grid=False) 
# p_grid_trans = np.empty_like(p_grid)
# p_grid_trans = (linear_distort @ (scale * (p_grid - 32.)).T).T + 32 + translation

# of_interp = interpolate_spacetime(interp_left, t, grid_size=20, x=p_grid_trans[:, 0], y=p_grid_trans[:, 1])


# ax[1].imshow(of_interp[5, :, :])
# ax[1].set_title("Distorted")



In [248]:
x = np.linspace(0, 64, 20 + 2, dtype=np.float32)[1:-1]
y = np.linspace(0, 64, 20 + 2, dtype=np.float32)[1:-1]

tt, xx, yy = np.meshgrid(t, x, y)

txy_grid = np.concatenate((tt.reshape(-1, 1), xx.reshape(-1, 1), yy.reshape(-1, 1)), axis=1)

In [17]:
# import matplotlib.pyplot as plt


# from src.features_calculator import (
#     new_concatenate_features,
#     get_augmentation_pars,
# )

# feature_array, all_timestamps, clip_transitions = rec._load_features(
#             clip_name, rec._of_params)

# # Add random sampling of some indicies

# n_clips = clip_transitions.shape[0] + 1
# clip_feature_array = np.split(feature_array, clip_transitions + 1, axis=0)
# clip_timestamps = np.split(all_timestamps, clip_transitions + 1, axis=0)


Number of clips: 2


(3490,)

In [1]:
from src.features_calculator import (
    new_concatenate_features,
    get_augmentation_pars,
    create_interpolater
)
from training.helper import get_augmentation_options
import matplotlib.pyplot as plt

interp_left, interp_right = create_interpolater(feature_array[:, :, 1], all_times)

n_clip_frames = clip_feature_array[iclip].shape[0]




all_times = (all_timestamps[0:10000] - all_timestamps[0]) / 1e9 
indc_times = [2, 500, 1000, 2000, 5000, 10000]

aug_params_options = get_augmentation_options()
aug_options = aug_params_options[0]

aug_params = get_augmentation_pars(aug_options)
aug_params["speed"] = 1
aug_params["translation"] = np.zeros((1, 2))
aug_params["scale"] = 1
aug_params["linear_distort"] = np.eye(2)

aug_feat = new_concatenate_features(
    interp_left,
    interp_right,
    n_clip_frames,
    of_params,
    indc_times[0],
    aug_params,
)

left = aug_feat[0, :675].reshape(3, 15, 15)
right = aug_feat[0, 675:].reshape(3, 15, 15)


aug_params = get_augmentation_pars(aug_options)
aug_params["speed"] = 1
aug_params["translation"] = [0, 10]
aug_params["scale"] = 1
aug_params["linear_distort"] = np.eye(2) + [[0, 0], [0, 0]]

aug_feat = new_concatenate_features(
    interp_left,
    interp_right,
    n_clip_frames,
    of_params,
    indc_times[0],
    aug_params,
)

left_aug = aug_feat[0, :675].reshape(3, 15, 15)
right_aug = aug_feat[0, 675:].reshape(3, 15, 15)


f, ax = plt.subplots(2, 2, figsize=(10, 5))

ax[0, 0].imshow(left[0, :, :], vmin=-3, vmax=1)
ax[0, 1].imshow(right[0, :, :], vmin=-3, vmax=1)
ax[1, 0].imshow(left_aug[0, :, :], vmin=-3, vmax=1)
ax[1, 1].imshow(right_aug[0, :, :], vmin=-3, vmax=1)

ModuleNotFoundError: No module named 'src'

In [28]:
of_params.layer_interval / 200

0.035

In [21]:
of_left = np.zeros([3490, 20, 20, 2])
of_left[:, 6:14, 6:14, :] = 1

of_right = np.zeros([3490, 20, 20, 2])
of_right[:, 6:14, 6:14, :] = 1

of_left = of_left.reshape(-1, 20*20, 2)
of_right = of_right.reshape(-1, 20*20, 2)

feature_array = np.concatenate((of_left, of_right), axis=1)

In [16]:
feature_array.shape

(3490, 800, 2)

In [119]:
 np.random.normal(1, 0)

1.0

In [121]:
np.random.normal(0, 0, 2)

array([0., 0.])

In [129]:
aug_params["translation"] * np.array([1, -1])

array([ 0, -5])

In [131]:
import itertools
std_speed = [0.0]
std_translation = [0, 1, 2, 3, 4, 5]
std_scale = [0.0, 0.05]
std_linear = [0.0]

options = itertools.product(
    std_speed,
    std_translation,
    std_scale,
    std_linear,
)

options = list(options)
# aug_params_options = sorted(set(AugParams(*option) for option in options))

# print(f"options {len(aug_params_options)}")


In [132]:
options

[(0.0, 0, 0.0, 0.0),
 (0.0, 0, 0.05, 0.0),
 (0.0, 1, 0.0, 0.0),
 (0.0, 1, 0.05, 0.0),
 (0.0, 2, 0.0, 0.0),
 (0.0, 2, 0.05, 0.0),
 (0.0, 3, 0.0, 0.0),
 (0.0, 3, 0.05, 0.0),
 (0.0, 4, 0.0, 0.0),
 (0.0, 4, 0.05, 0.0),
 (0.0, 5, 0.0, 0.0),
 (0.0, 5, 0.05, 0.0)]

In [52]:
from random import choices, sample

In [54]:
a = [1, 2, 3, 4, 5]

sample(a, k=5)

[2, 1, 5, 4, 3]

In [21]:
def _get_blink_labels(clip_name, timestamps):

    blink_df = rec._load_gt_labels(clip_name)

    on_start = blink_df[blink_df["label"] == "onset"]["start_ts"]
    on_start_idc = np.where(np.isin(timestamps, on_start))[0]

    on_end = blink_df[blink_df["label"] == "onset"]["end_ts"]
    on_end_idc = np.where(np.isin(timestamps, on_end))[0]

    off_start = blink_df[blink_df["label"] == "offset"]["start_ts"]
    off_start_idc = np.where(np.isin(timestamps, off_start))[0]

    off_end = blink_df[blink_df["label"] == "offset"]["end_ts"]
    off_end_idc = np.where(np.isin(timestamps, off_end))[0]

    blink_vec = np.zeros_like(timestamps)

    for onset, offset in zip(on_start_idc, on_end_idc):
        blink_vec[onset:offset] = 1

    for onset, offset in zip(off_start_idc, off_end_idc):
        blink_vec[onset:offset] = 2

    blink_labels = dict()
    blink_labels["onset_indices"] = np.where(blink_vec == 1)[0]
    blink_labels["offset_indices"] = np.where(blink_vec == 2)[0]
    blink_labels["blink_indices"] = np.where(blink_vec == 2)[0]

    return blink_labels

In [24]:
timestamps = rec._get_timestamps(clip_name)
blink_labels = _get_blink_labels(clip_name, timestamps)

In [27]:
def _find_indices(
    self,
    feature_array,
    clip_name: str,
    timestamps: np.ndarray,
    n_frames: int,
    bg_ratio: int,
    half: bool,
) -> T.Tuple[np.ndarray, np.ndarray, np.ndarray]:

    blink_labels = self._get_blink_labels(clip_name, timestamps)

    onset_indices = blink_labels["onset_indices"]
    offset_indices = blink_labels["offset_indices"]
    blink_indices = blink_labels["blink_indices"]

    # n_samples = np.min(0.2 * len(onset_indices), 20)
    # onset_indices = choices(onset_indices, k=bg_ratio)
    # n_samples = np.min(0.2 * len(offset_indices), 20)
    # offset_indices = choices(offset_indices, k=bg_ratio)

    # blink_indices = list(set(onset_indices) | (set(offset_indices)))
    # blink_indices.sort()

    bg_indices = self._get_background_indices(blink_indices, n_frames, bg_ratio)
    pulse_indices = np.where(abs(np.mean(feature_array, axis=1))[:, 1] > 0.075)[0]
    all_indices = np.hstack([blink_indices, bg_indices, pulse_indices])
    all_indices = np.unique(all_indices)
    all_indices = all_indices.astype(np.int64)

    return np.array(onset_indices), np.array(offset_indices), all_indices


(array([ 12126,  12127,  12128,  12129,  12130,  12131,  12132,  12133,
         12134,  12135,  12136,  12137,  12138,  12139,  12140,  12141,
         12248,  12249,  12250,  12251,  12252,  12253,  12254,  12255,
         12256,  12257,  12258,  12259,  12260,  12261,  12262,  12263,
         12321,  12322,  12323,  12324,  12325,  12326,  12327,  12328,
         12329,  12330,  12331,  12332,  12333,  12334,  12431,  12432,
         12433,  12434,  12435,  12436,  12437,  12438,  12439,  12440,
         12441,  12442,  12443,  12444,  12445,  12446,  12447,  12448,
        320832, 320833, 320834, 320835, 320836, 320837, 320838, 320839,
        320840, 320841, 321158, 321159, 321160, 321161, 321162, 321163,
        321164, 321165, 321166, 321481, 321482, 321483, 321484, 321485,
        321486, 321487, 321488, 321489, 321985, 321986, 321987, 321988,
        321989, 321990, 321991, 321992, 321993, 321994, 321995, 321996,
        321997, 321998, 321999, 322000, 322001, 322002, 322003, 

In [49]:
onset, offset, all = rec._find_indices(
    feature_array,
    clip_name,    
    timestamps,
    n_frames,
    bg_ratio=3,
    half=False,)

In [50]:
gt_labels = np.full(n_frames, 0)
if len(offset):
    gt_labels[offset] = 2
if len(onset):
    gt_labels[onset] = 1
gt_labels = gt_labels[all]


IndexError: index 12142 is out of bounds for axis 0 with size 3490

In [51]:
offset

array([ 12142,  12143,  12144,  12145,  12146,  12147,  12148,  12149,
        12150,  12151,  12152,  12153,  12154,  12155,  12156,  12157,
        12158,  12159,  12160,  12161,  12162,  12163,  12164,  12165,
        12166,  12167,  12168,  12169,  12170,  12171,  12172,  12264,
        12265,  12266,  12267,  12268,  12269,  12270,  12271,  12272,
        12273,  12274,  12275,  12276,  12277,  12278,  12279,  12280,
        12281,  12282,  12283,  12284,  12285,  12335,  12336,  12337,
        12338,  12339,  12340,  12341,  12342,  12343,  12344,  12345,
        12346,  12347,  12348,  12349,  12350,  12351,  12352,  12353,
        12354,  12355,  12356,  12357,  12358,  12359,  12360,  12449,
        12450,  12451,  12452,  12453,  12454,  12455,  12456,  12457,
        12458,  12459,  12460,  12461,  12462,  12463,  12464,  12465,
        12466,  12467,  12468,  12469,  12470,  12471,  12472, 320842,
       320843, 320844, 320845, 320846, 320847, 320848, 320849, 320850,
      

In [51]:
from src.features_calculator import create_grids
from scipy.interpolate import griddata



p_grid = create_grids((64, 64), 20, full_grid=True)
feature_array = clip_feature_array[0]

left = feature_array[:, :400, :].transpose(1, 0, 2)
create_grids((20, 20), 20, True)

right = feature_array[:, 400:, :].transpose(1, 0, 2)

sub_grid = create_grids((64, 64), 6, False)

feat_left = griddata(p_grid, left, sub_grid, method="linear").transpose(1, 0, 2)
feat_right = griddata(p_grid, right, sub_grid, method="linear").transpose(1, 0, 2)

# f, ax = plt.subplots(1, 2, figsize=(10, 5))
# ax[0].imshow(feat.reshape(-1, 16, 16, 2)[20, :, :, 0])
# ax[1].imshow(left.transpose(1, 0, 2).reshape(-1, 20, 20, 2)
# [20, :, :, 0])




In [53]:
np.concatenate((feat_left, feat_right), axis=1).shape

(558, 32, 2)

In [64]:
all_aug_samples = {}


In [65]:
all_aug_samples["abc"]

KeyError: 'abc'

In [26]:
import numpy as np

np.random.normal(0, 0.5, 2)

array([-0.05560818,  0.20667585])